In [ ]:
from pymongo import MongoClient
import pandas as pd

client = MongoClient('127.0.0.1',27017) #is assigned local port
dbName = "TwitterDump2" # set-up a MongoDatabase
db = client[dbName]
collName = 'colTest2' # here we create a collection
collection = db[collName] #  This is for the Collection  put in the DB

cursor=db['colTest2'].find({})
all_tweets=pd.DataFrame(list(cursor))
print(all_tweets.iloc[0])
#del all_tweets['_id']

ServerSelectionTimeoutError: ignored

In [ ]:
dbs = MongoClient('127.0.0.1',27017).list_database_names()
print(dbs)

In [ ]:
print(all_tweets.shape)
print(all_tweets.shape[0])

In [ ]:
print(all_tweets.iloc[(all_tweets.shape[0]-1)])

In [ ]:
located_tweets=all_tweets[all_tweets['location'].notnull()]
UK_tweets=located_tweets[located_tweets['location'].str.contains("United Kingdom")]
print(UK_tweets.shape)

In [ ]:
Loc_UK = [-10.392627, 49.681847, 1.055039, 61.122019]
geotweets=all_tweets[all_tweets['place_coordinates'].notnull()]
print(geotweets.shape)

Coord_UK=geotweets.copy()
droplist=[]

for i in range(len(Coord_UK)) :
    #print(Coord_UK.iloc[i]['place_coordinates'])
    if Coord_UK.iloc[i]['place_coordinates'][0][0][0]<(-10.392627) or Coord_UK.iloc[i]['place_coordinates'][0][0][0]>(1.055039)or Coord_UK.iloc[i]['place_coordinates'][0][0][1]>(61.122019)or Coord_UK.iloc[i]['place_coordinates'][0][0][1]<(49.681847):
        droplist.append(Coord_UK.iloc[i].name)
        
for x in droplist:
    Coord_UK.drop(x,inplace=True)

In [ ]:
print(Coord_UK.shape)


In [ ]:
retweets=all_tweets.copy()
droplist=[]

for i in range(len(retweets)) :
    
    if not retweets.iloc[i].get('retweet_status'):
        droplist.append(retweets.iloc[i].name)

        
for x in droplist:
    retweets.drop(x,inplace=True)
    
print(retweets.shape)

In [ ]:
#install extra modules using conda
import spacy
nlp = spacy.load('en_core_web_sm', disable=['ner'])




def spacy_tokenize(string):
    tokens = list()
    doc = nlp(string)
    for token in doc:
        tokens.append(token)
    return tokens

# This tokenizes the body posts and creates vector of tokens for each post.
# Or rather the text
all_posts_tokenized = Coord_UK.text.apply(spacy_tokenize)

import itertools
# A single variable with the (flattened) tokens from all posts.
flat_tokens = list(itertools.chain.from_iterable(all_posts_tokenized))

def normalize(tokens):
    normalized = list()
    for token in tokens:
        if (token.is_alpha or token.is_digit):
            lemma = token.lemma_.lower().strip() if token.lemma_ != "-PRON-" else token.lower_
            normalized.append(lemma)
    return normalized

def tokenize_normalize(string):
    return normalize(spacy_tokenize(string))

In [ ]:
from itertools import islice

# Parallel arrays of the post keys and values.
post_vals = list()


for post in Coord_UK.T.iteritems():
    post_vals.append(post[1].text)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
#arrange them by text frequency/ inverse document frquency

vectorizer = TfidfVectorizer(tokenizer=tokenize_normalize, binary=True,stop_words={'english'})
terms = vectorizer.fit_transform(post_vals)
print(terms)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeansSum_of_squared_distances = []
K = range(2,10)for k in K:
    km = KMeans(n_clusters=k, max_iter=200, n_init=10)
    km = km.fit(X)
    Sum_of_squared_distances.append(km.inertia_)plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()
#thanks to towards data science for elbow method code